In [1]:
import torch
import torch.nn as nn
# from torch import optim
# from torch.nn import utils
import matplotlib.pyplot as plt

In [2]:
# Step 1

def objective_function(d , s):
    f = (s[0,0]**2) + (s[0,1]**2) + (d**2)
    return f

def constraints(d, s):
    h = torch.zeros(1,2)
    h[0,0] = (s[0,0]**2)/4 + (s[0,1]**2)/5 + (d**2)/25 - 1
    h[0,1] = s[0,0] + s[0,1] - d
    h = h.T
    #print(f'h = {h}')
    return h

# step 3

def reduced_gradient(d, s):
  df_dd = 2*d

  df_ds = torch.zeros(1,2)
  df_ds[0,0] = 2*s[0,0]
  df_ds[0,1] = 2*s[0,1]

  dh_ds = torch.zeros(2,2)
  dh_ds[0,0] = (s[0,0])/2
  dh_ds[0,1] = 2*(s[0,1])/5
  dh_ds[1,0] = 1
  dh_ds[1,1] = 1

  dh_dd = torch.zeros(2,1)
  dh_dd[0,0] = 2*(d/25)
  dh_dd[1,0] = -1

  inv_dhds_dhdd = torch.linalg.solve(dh_ds,dh_dd)
  #print(f'inv_dhds = {inv_dhds_dhdd}')
  d_subtract_term = df_dd - torch.matmul(df_ds, inv_dhds_dhdd)
  #print(dfddstep)
  return d_subtract_term


# step 4.3
def step4_3_term(d,s):
  dh_ds = torch.zeros(2,2)
  dh_ds[0,0] = (s[0,0])/2
  dh_ds[0,1] = 2*(s[0,1])/5
  dh_ds[1,0] = 1
  dh_ds[1,1] = 1

  dh_dd = torch.zeros(2,1)
  dh_dd[0,0] = 2*(d/25)
  dh_dd[1,0] = -1

  df_dd = 2*d
  invterm = torch.linalg.solve(dh_ds,dh_dd)
  step4_3_term = invterm*(df_dd.T)
  return step4_3_term.T


def levenberg_marquardt(d, s):
    dh_ds = torch.zeros(2,2)
    dh_ds[0,0] = (s[0,0])/2
    dh_ds[0,1] = 2*(s[0,1])/5
    dh_ds[1,0] = 1
    dh_ds[1,1] = 1
    h = constraints(d,s)
    d2fds2 = torch.matmul(dh_ds.T,dh_ds)
    lmbda = 0.1
    I = torch.eye(2)
    term = d2fds2 + lmbda*I
    #print(f'term = {term}')
    invterm = torch.linalg.solve(term,dh_ds.T)
    #print(f'invterm = {invterm}')
    #print(s)
    step_4_4_term = torch.matmul(invterm, h)
    step_4_4_term = step_4_4_term.T
    return step_4_4_term

In [3]:
max_iter = 100

# Step 2
#x = torch.tensor([[1, 1.56, 2.56]])
#x = x.T
tol = 10e-3
d = torch.tensor([[1]])
s =  torch.tensor([[ 1.81791441, -0.81791441]])


# Step 3
grad = reduced_gradient(d,s)
norm = torch.mean(grad**2)
#print(f"Norm = {norm}")
#for _ in range (max_iter):
delta = 1
# Step 4
while norm > tol:

  h = constraints(d,s)
  #print(f'h = {h}')

  # Step 4.1
  alpha = 0.01
  b = 0.5
  t = 0.3

  dk_sub_grad = d - alpha*grad
  sk_plus_term = s + alpha*(step4_3_term(d,s))
  f = objective_function(dk_sub_grad, sk_plus_term)
  #print(f'f alp = {f}')

  phi = objective_function(d, s)
  #print(f'phi alp = {phi}')

  # while f>phi:
  #   alpha = b*alpha
  #   dk_sub_grad = d - alpha*grad
  #   sk_plus_term = s + alpha*(step4_3_term(d,s))
  #   f = objective_function(dk_sub_grad, sk_plus_term)
  #   #print(f'f alp = {f}')

  #   phi = objective_function(d, s)
    #print(f'phi alp = {phi}')

  # Step 4.2

  dk = dk_sub_grad

  # Step 4.3
  sk0term = step4_3_term(d,s)
  sk_not = sk_plus_term

  # Step 4.4
  h = constraints(dk,sk_not)
  h_norm1 = torch.sqrt(h[0,0]**2)
  h_norm2 = torch.sqrt(h[1,0]**2)

#remove this condition for the code to work
  # while h_norm1 > 10e-2 or h_norm2 > 10e-2:
  #   print(f'Norm1 = {h_norm1}')
  #   print(f'Norm2 = {h_norm2}')

  #   lmterm = levenberg_marquardt(dk, sk_not)
  #   sk = sk_not - 0.001*lmterm
  #   h = constraints(dk,sk)
  #   h_norm = torch.mean(h**2)
  #   sk_not = sk
  #   print(f'sK = {sk}')

  sk =sk_not
  # Step 4.5
  gradk = reduced_gradient(dk,sk)
  delta = ((gradk-grad)**2)

  norm = torch.mean(gradk**2)
  #print(f'Norm = {norm}')
  grad = gradk
  d = dk
  s = sk
print(f'objective function = {f[0,0]}')
h = constraints(d,s)
print(f'h = [{h[0,0]}, {h[1,0]}]')
print(f'x1 = {s[0,0]}')
print(f'x2 = {s[0,1]}')
print(f'x3 = {d[0,0]}')

objective function = 4.811403274536133
h = [0.032567620277404785, -0.7518693208694458]
x1 = 1.276801586151123
x2 = -1.7638112306594849
x3 = 0.2648596465587616
